# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [1]:
import getpass
myname = getpass.getuser().split('-')[1]
myname
MYTABLE=myname + '_data'


In [2]:
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [3]:
host = 'slucor2022-instance-1.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2022'
password = 'SLUcor2022'
database = 'hds5210'
conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)

In [4]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [5]:
url = "https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326"
my_data = pd.read_csv(url)

In [6]:
my_data.to_sql(myname + '_data', conn, index=True, if_exists="replace")

In [7]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
dxyz.shape

(323, 22)

In [8]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
assert(dxyz.shape == (323,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Business_ZIP_Code` field and `population`'s `Zip` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [9]:

tables = {}

for n,t in pd.read_sql_query("SHOW TABLES", conn).iterrows():
    name = str(t['Tables_in_hds5210'])
    print(name)
    tables[name] = pd.read_sql("SELECT * FROM "+ name, conn)
tables['population']
zip_codes = pd.read_sql_query(""" SELECT max(p.Population), d.NPI
  From
    population p INNER JOIN
    surya_data d ON p.Zip = d.Business_ZIP_Code
    """, conn)
zip_codes
zip_codes['NPI'] = zip_codes['NPI'].astype(str)
answer = zip_codes['NPI'].values.tolist()

assert(type(answer) == list)
assert(answer == ['1194016923'])


asahu123_data
comp_tot_pay
damonchen_data
data
ecdmdphd_data
ed_visits
havenaspen_data
havenaspen_ipv
hospital_services
hospitals
kchowdary110_data
lillyeversman_data
paulboal_data
payment_data
payment_data1
population
rasalt_data
rasalt_data_tmp
rasalt_data_tmp1
rasalt_data_tmp2
sahaja326_data
sairupesh24_data
sairupesh24_data1
shiv80_data
surya_data
svelicheti_data
system_affiliations
tot_pay_made
total_per_population
vikas060_data
vikas060data
vineethmunigati_data
vineethmunigati_data_data


In [10]:
answer

['1194016923']

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

## First using SQL

In [11]:
payment = pd.read_sql_query("""
  SELECT a.Medicaid_EP_Hospital_type, sum(a.total_payments) as payments, a.Business_ZIP_Code
  FROM
    surya_data a
  GROUP BY
     a.Medicaid_EP_Hospital_Type, a.Business_ZIP_Code
""", conn)

In [12]:
payment.to_sql('payment_data1', conn, index = False)

ValueError: Table 'payment_data1' already exists.

In [ ]:
answer = pd.read_sql_query("""
  SELECT a.Medicaid_EP_Hospital_Type, AVG(a.payments/p.Population) as Avg_Pay_per_Capita
  from
    population p JOIN
    payment_data a ON p.Zip = a.Business_ZIP_Code
  Group by
    a.Medicaid_EP_Hospital_Type
""", conn)

In [ ]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 106.583)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 329.551)


In [ ]:
answer

## Now using Pandas instead of SQL

In [ ]:
payment_zip = pd.pivot_table(dataframe,
            index = ['Business_ZIP_Code', 'Medicaid_EP_Hospital_Type'],
                            values = 'total_payments',
                            aggfunc = np.sum).reset_index()
total = payment_zip.merge(tables['population'], left_on = 'Business_ZIP_Code', right_on = 'Zip')
total['per_capita'] = (total['total_payments']/total['Population'])
avg_per_capita = total.groupby(['Medicaid_EP_Hospital_Type'])['per_capita'].agg(Avg_Pay_per_Capita = ('mean')).reset_index()
avg_per_capita



---



## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [ ]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git pull
    !git add week15_assignment.ipynb
    !git commit -a -m "Submitting the week 15 programming assignment"
    !git push
else:
    print('''
    
OK. We can wait.
''')